In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 라이브러리와 app 데이터 세트 로딩

import numpy as np
import pandas as pd
import time  # 날짜/시간을 위한 라이브러리
import gc  # Garbage Collection  --> 파이썬 애플리케이션 메모리 관리 == 구동 중에 필요하지 않은 메모리를 관리
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

#import warning
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


In [ ]:
# 화면의 크기 설정

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
# listdir를 사용하여 디렉터리 리스트 확인

print(os.listdir("../input/home-credit-default-risk"))

## Application_train 과 Application_test 데이터 read_csv

In [ ]:
# Application_train 데이터 feature 확인
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

In [ ]:
# Testing data features 확인
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

In [ ]:
# training 데이터와 testing 데이터를 한번에 shape 다시 확인하여 비교
# app_train.shape, app_test.shape #실행해도 됨

print('Training data shape: ', app_train.shape)
print('Testing data shape: ', app_test.shape)

In [ ]:
app_train.shape, app_test.shape

In [ ]:
# training 데에터의 컬럼 확인
app_train.columns.values

## Target 컬럼의 분포를 시각화를 통해 확인
    - Target은 예측해야하는 값임
    - 0이면 제때 대출금 상환 가능한것, 1이면 상황이 어려운것을 의미
    - target 0 과 target 1에 따라 어떤 histogram을 갖고 있는지 확인할 필요성이 있음

In [ ]:
# Target의 value_counts 확인
app_train['TARGET'].value_counts()

In [ ]:
app_train.shape

In [ ]:
# 차이가 있어 보이는데...app_train.shape는 (307511, 122) --> 전체 데이터에서 target의 비율 확인 -->  282686 / 307511

app_train['TARGET'].value_counts()/app_train.shape[0]  

In [ ]:
app_train['TARGET'].astype(int).plot.hist()

# 불균형 한 데이터임 --> 대출을 상환할 수 있는 0값이 1보다 훨씬 많은 imbalanced data
# --> 구분할 각 분류에 해당되는 데이터의 비율이 반반이 아닌 경우 훈령 데이터내 비율이 높은 분류 쪽으로 결과를 분류하는 모델을 만들수 있음 --> class imbalance(클래스 불균형)
# --> 1) 가중치(weight)를 조절, or 2) 더 많은 비용(cost)을 부과. or 3) 훈련 데이터를 직접 조절(SMOTE)
# --> 모델링 할때 살펴보겠음

In [ ]:
# Target 값에 따른 AMT_INCOME_TOTAL(소득)값 분포 확인하기 위해 소득 데이터 확인

# 1) AMT_INCOME_TOTAL(소득) 시각화

app_train['AMT_INCOME_TOTAL'].hist()

#plt.hist(app_train['AMT_INCOME_TOTAL'])

In [ ]:
# 2) sns에서 distplot을 이용하면, histogram 그릴수 있어요

sns.distplot(app_train['AMT_INCOME_TOTAL'])

In [ ]:
# 3) 이렇게 scale 큰 데이터를 확인 할때 boxplot 사용하는 것이 좋음
# # 4분위를 보여줌
sns.boxplot(app_train['AMT_INCOME_TOTAL'])

In [ ]:
# AMT_INCOME_TOTAL이 1000000 이하인 값에 대한 분포도
# filtering 후 histogram 표현

condition_1 = app_train['AMT_INCOME_TOTAL'] < 1000000
app_train[app_train['AMT_INCOME_TOTAL'] < 1000000] # 조건에 해당되는 모든 행 출력된

In [ ]:
# 조건: 소득 < 1000000 추출

app_train[app_train['AMT_INCOME_TOTAL'] < 1000000]

In [ ]:
#위 코드 뒤에 추가하여 hist 표현

app_train[app_train['AMT_INCOME_TOTAL'] < 1000000]['AMT_INCOME_TOTAL'].hist()

# app_train[condition_1]['AMT_INCOME_TOTAL'].hist() # condition_1 에 저장하였던 조건을 이용하여 표현 가능

In [ ]:
# 이산형을 연속형으로 적용해줌(KDE)
sns.distplot(app_train[condition_1]['AMT_INCOME_TOTAL'])

### TARGET 값에 따른 AMT_INCOME_TOTAL값 분포도 비교
    - distplot과 violinplot 시각화
    - plt.subplots() 기반으로 seaborn의 distplot과 violinplot으로 분포도 비교 시각화

In [ ]:
# cond1 = 조건 1
# cond0 = 조건 2
cond1 = (app_train['TARGET'] == 1)
cond0 = (app_train['TARGET'] == 0)
cond_amt = (app_train['AMT_INCOME_TOTAL'] < 500000)

# 위 조건들 결합

 # 논리 연산자 1조건 & 2조건  ==> TRUE & TRUE ==> TRUE; FALSE & TRUE ==> False
sns.distplot(app_train[cond0 & cond_amt]['AMT_INCOME_TOTAL'], label='0', color='blue') # 두 조건이 true 일때 (TARGET==0 과 AMT_INCOME_TOTAL < 500000)에 해당 데이터
sns.distplot(app_train[cond1 & cond_amt]['AMT_INCOME_TOTAL'], label='1', color='red') # 두 조건이 true 일때

## 만약 kde를 보고싶지 않으면 아래 코드에 kde=False를 추가해주면 됨

In [ ]:
# violonplot
# x 는 category plot (비교하고자하는 값), y는 분포들을 보려는 컬럼 값

sns.violinplot(x='TARGET', y='AMT_INCOME_TOTAL', data=app_train[cond_amt])

In [ ]:
# 한번에 비교하여 그리기

fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)

sns.violinplot(x='TARGET', y='AMT_INCOME_TOTAL', data=app_train[cond_amt], ax=ax[0][0])
sns.distplot(app_train[cond0 & cond_amt]['AMT_INCOME_TOTAL'], label='0', color='blue', ax=ax[0][1])
sns.distplot(app_train[cond1 & cond_amt]['AMT_INCOME_TOTAL'], label='1', color='red', ax=ax[0][1])

In [ ]:
# 함수로 만들기

def show_column_hist_by_target(df, column, is_amt=False):
    
    cond0 = (df['TARGET'] == 0)
    cond1 = (df['TARGET'] == 1)
    
    fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)
    # is_amt 가 True면 < 500000 조건으로 필터링
    cond_amt = True
    if is_amt:
        cond_amt = df[column] < 500000
    
    sns.violinplot(x='TARGET', y=column, data=df[cond_amt], ax=ax[0][0])
    sns.distplot(df[cond0 & cond_amt][column], ax=ax[0][1], label='0', color='blue')
    sns.distplot(df[cond1 & cond_amt][column], ax=ax[0][1], label='1', color='red')

show_column_hist_by_target(app_train, 'AMT_INCOME_TOTAL', is_amt=True)

### app_train과 app_test를 합쳐서 한번에 데이터 preprocessing 수행 
    - 기본 데이터셋을 그대로 활용

In [ ]:
app_train.shape, app_test.shape

In [ ]:
apps = pd.concat([app_train, app_test])
apps.shape

In [ ]:
apps['TARGET'].value_counts(dropna=False)

### Object feature들을 Label Encoding
    -  Label encoding : 범주형 변수의 개별값을 숫자로 바꿔주는 방법. 컬럼을 새로 생성하지 않음
    -  여성/남성 처럼 범주형 변수의 값이 두개일경우는 Label encoding을 사용해도 무관하지만, 그 이상일경우는 One-hot encoding을 사용하는것이 좋음

In [ ]:
apps.info()

In [ ]:
apps.dtypes.index

In [ ]:
object_columns = apps.dtypes[apps.dtypes == 'object'].index.tolist()
object_columns

In [ ]:
apps['CODE_GENDER'] = pd.factorize(apps['CODE_GENDER'])[0]

In [ ]:
# Label Encoding으로 인해 object가 16개에서 15개로 줄었음
apps.info()

In [ ]:
# 모든 컬럼 변경
object_columns = apps.dtypes[apps.dtypes == 'object'].index.tolist()

for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0]

### Null값 일괄 변환

In [ ]:
apps.isnull().sum().head(100)

In [ ]:
# -999로 모든 컬럼들의 Null값 변환
apps = apps.fillna(-999)

In [ ]:
apps.isnull().sum().head(100)

### 학습 데이터와 테스트 데이터 다시 분리

In [ ]:
app_train = apps[apps['TARGET'] != -999]
app_test = apps[apps['TARGET'] == -999]
app_train.shape, app_test.shape

In [ ]:
app_test = app_test.drop('TARGET', axis=1, inplace=False)
app_test.shape

### 학습 데이터를 검증 데이터로 분리하고 LGBM Classifier로 학습 수행
    - 피처용 데이터와 타겟 데이터 분리
    - 학습용/검증용 데이터 세트 분리

In [ ]:
ftr_app = app_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = app_train['TARGET']


In [ ]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
eval_metric= 'auc', verbose= 100, early_stopping_rounds= 50)

### Feature importance 시각화

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

### 학습된 Classifier를 이용하여 테스트 데이터을 예측하고 결과를 Kaggle로 Submit

In [ ]:
clf.predict_proba(app_test.drop(['SK_ID_CURR'], axis=1))

In [ ]:
pred = clf.predict_proba(app_test.drop(['SK_ID_CURR'], axis=1))[:, 1]

app_test['TARGET'] = pred
app_test['TARGET'].head()

In [ ]:
app_test[['SK_ID_CURR', 'TARGET']].to_csv('app_baseline_01.csv', index=False)